In [1]:
# NB: This is for spark running on parquet files converted from
#     baconbits skims. This is a prototype, there is lots of boilerplate.
#     We're making it better :-)

import pyspark.sql
import os
import sys
from pyarrow.compat import guid
from coffea.processor.spark.detail import _spark_initialize

# The following line is necessary because we're working in a
# virtualenv. Without it, executors will use the wrong interpreter!
#os.environ['PYSPARK_PYTHON'] = sys.executable

#    .config("spark.driver.extraClassPath","/home/cms.lgray/sparkMeasure/target/scala-2.12/spark-measure_2.12-0.14-SNAPSHOT.jar") \
#    .master('local[*]') \
#.master('spark://cmsspark-submit.fnal.gov:7077') \
#spark://cmsspark-submit.fnal.gov:7077
conf = pyspark.sql.SparkSession.builder \
    .appName('pancakes-spark-%s' % guid()) \
    .master('spark://cmsspark-submit.fnal.gov:7077') \
    .config('spark.driver.cores',8) \
    .config('spark.driver.memory','10g') \
    .config('spark.executor.memory', "20g") \
    .config('spark.debug.maxToStringFields',1000) \
    .config('spark.executorEnv.XRD_RUNFORKHANDLER','1') \
    .config('spark.executorEnv.X509_PROXY',"%s" % os.environ['X509_USER_PROXY']) \
    .config('spark.files','file://%s' % os.environ['X509_USER_PROXY']) \
    .config('spark.sql.parquet.mergeSchema', 'true') \
    .config('spark.sql.execution.arrow.enabled','true') \
    .config('spark.blockManager.port', "8000") \
    .config('spark.sql.execution.arrow.pyspark.enabled','true') \
    .config('spark.sql.execution.arrow.maxRecordsPerBatch', 200000) \
    .config('spark.hadoop.fs.xrootd.read.buffer', 75 * 1048576) \
    .config('spark.hadoop.fs.xrootd.readahead.range', 50 * 1048576) \
    .config('spark.driver.extraClassPath','file:///opt/hadoop-xrootd/hadoop-xrootd-1.0.4-jar-with-dependencies.jar') \
    .config('spark.executor.extraClassPath','file:///opt/hadoop-xrootd/hadoop-xrootd-1.0.4-jar-with-dependencies.jar')
session = _spark_initialize(config=conf,log_level='WARN',laurelin_version='0.4.2-SNAPSHOT')
sc = session.sparkContext
sc.setLogLevel("WARN")
spark = session

#from sparkmeasure import TaskMetrics
#taskmetrics = TaskMetrics(spark)

partitionsize = 200000
thread_workers = 16


In [2]:
nano_qcdht_700to1000 = """/store/mc/RunIIFall17NanoAODv5/QCD_HT700to1000_TuneCP5_13TeV-madgraph-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano1June2019_102X_mc2017_realistic_v7-v1/50000/6F725029-DF13-7149-A458-7A8DE2296539.root
/store/mc/RunIIFall17NanoAODv5/QCD_HT700to1000_TuneCP5_13TeV-madgraph-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano1June2019_102X_mc2017_realistic_v7-v1/50000/AAF874D3-2C58-B84C-8389-A4C4902E5DF0.root
/store/mc/RunIIFall17NanoAODv5/QCD_HT700to1000_TuneCP5_13TeV-madgraph-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano1June2019_102X_mc2017_realistic_v7-v1/50000/0D4E23AF-951E-BD44-9AE0-9415A1B97E7F.root
/store/mc/RunIIFall17NanoAODv5/QCD_HT700to1000_TuneCP5_13TeV-madgraph-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano1June2019_102X_mc2017_realistic_v7-v1/50000/685B4F02-EC4E-1A4E-BF73-44CA1F917B3A.root
/store/mc/RunIIFall17NanoAODv5/QCD_HT700to1000_TuneCP5_13TeV-madgraph-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano1June2019_102X_mc2017_realistic_v7-v1/50000/CB9508D8-35F5-2C45-91B9-B29B18E4BE15.root
/store/mc/RunIIFall17NanoAODv5/QCD_HT700to1000_TuneCP5_13TeV-madgraph-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano1June2019_102X_mc2017_realistic_v7-v1/50000/F34CA315-73BA-1845-9F0D-FF9D4FBAEE5B.root
/store/mc/RunIIFall17NanoAODv5/QCD_HT700to1000_TuneCP5_13TeV-madgraph-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano1June2019_102X_mc2017_realistic_v7-v1/50000/E192E69A-07CB-E445-AE69-8CD9ABE872AA.root
/store/mc/RunIIFall17NanoAODv5/QCD_HT700to1000_TuneCP5_13TeV-madgraph-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano1June2019_102X_mc2017_realistic_v7-v1/50000/C824F0EB-5323-344F-B5AC-2A10D98E56AA.root
/store/mc/RunIIFall17NanoAODv5/QCD_HT700to1000_TuneCP5_13TeV-madgraph-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano1June2019_102X_mc2017_realistic_v7-v1/50000/BD3E916E-EBB4-4C44-BAFE-0C3885EDF328.root
/store/mc/RunIIFall17NanoAODv5/QCD_HT700to1000_TuneCP5_13TeV-madgraph-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano1June2019_102X_mc2017_realistic_v7-v1/50000/1DF82CEE-72D4-A84B-AC12-6CCB8EFF68FA.root
/store/mc/RunIIFall17NanoAODv5/QCD_HT700to1000_TuneCP5_13TeV-madgraph-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano1June2019_102X_mc2017_realistic_v7-v1/50000/9983462D-25CD-994F-A2C0-E74F064F31E5.root
/store/mc/RunIIFall17NanoAODv5/QCD_HT700to1000_TuneCP5_13TeV-madgraph-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano1June2019_102X_mc2017_realistic_v7-v1/50000/95C9AE79-DC06-C643-9A57-EB7756228783.root
/store/mc/RunIIFall17NanoAODv5/QCD_HT700to1000_TuneCP5_13TeV-madgraph-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano1June2019_102X_mc2017_realistic_v7-v1/50000/2AD2E6E0-2577-824C-918D-64F30E62F2F8.root
/store/mc/RunIIFall17NanoAODv5/QCD_HT700to1000_TuneCP5_13TeV-madgraph-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano1June2019_102X_mc2017_realistic_v7-v1/50000/D325BAEB-8E0F-784B-A666-FC827A6978AB.root
/store/mc/RunIIFall17NanoAODv5/QCD_HT700to1000_TuneCP5_13TeV-madgraph-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano1June2019_102X_mc2017_realistic_v7-v1/50000/EFB5E508-CAEB-3A40-ADA0-54D06892517F.root
/store/mc/RunIIFall17NanoAODv5/QCD_HT700to1000_TuneCP5_13TeV-madgraph-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano1June2019_102X_mc2017_realistic_v7-v1/50000/57DD0602-9A75-7548-8127-49FCFB252422.root
/store/mc/RunIIFall17NanoAODv5/QCD_HT700to1000_TuneCP5_13TeV-madgraph-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano1June2019_102X_mc2017_realistic_v7-v1/50000/02730FBB-AA2A-B540-99E9-223577DF4FEB.root
/store/mc/RunIIFall17NanoAODv5/QCD_HT700to1000_TuneCP5_13TeV-madgraph-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano1June2019_102X_mc2017_realistic_v7-v1/50000/56574349-74DE-7D49-B358-7560C8BF164A.root
/store/mc/RunIIFall17NanoAODv5/QCD_HT700to1000_TuneCP5_13TeV-madgraph-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano1June2019_102X_mc2017_realistic_v7-v1/50000/571A7325-E474-7242-9A18-A489E2B7D30B.root
/store/mc/RunIIFall17NanoAODv5/QCD_HT700to1000_TuneCP5_13TeV-madgraph-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano1June2019_102X_mc2017_realistic_v7-v1/50000/13E65A26-E054-854F-8090-A02EED9FD12C.root
/store/mc/RunIIFall17NanoAODv5/QCD_HT700to1000_TuneCP5_13TeV-madgraph-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano1June2019_102X_mc2017_realistic_v7-v1/50000/EEB5DAAC-E68C-0A45-A183-07A9DA9176B7.root
/store/mc/RunIIFall17NanoAODv5/QCD_HT700to1000_TuneCP5_13TeV-madgraph-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano1June2019_102X_mc2017_realistic_v7-v1/50000/455D7FA7-EA57-4B4A-AFBD-81B9AB194765.root
/store/mc/RunIIFall17NanoAODv5/QCD_HT700to1000_TuneCP5_13TeV-madgraph-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano1June2019_102X_mc2017_realistic_v7-v1/50000/6A702983-25CE-4A4A-9423-0A2698F892E9.root
/store/mc/RunIIFall17NanoAODv5/QCD_HT700to1000_TuneCP5_13TeV-madgraph-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano1June2019_102X_mc2017_realistic_v7-v1/50000/8FE8C2AC-21D2-5D49-8B43-16D915FA2FA3.root
/store/mc/RunIIFall17NanoAODv5/QCD_HT700to1000_TuneCP5_13TeV-madgraph-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano1June2019_102X_mc2017_realistic_v7-v1/50000/D224264E-95EE-C74D-81DB-4A8D1731426F.root
/store/mc/RunIIFall17NanoAODv5/QCD_HT700to1000_TuneCP5_13TeV-madgraph-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano1June2019_102X_mc2017_realistic_v7-v1/50000/1EB8474A-F30D-2C44-84B0-C474A8FDA650.root
/store/mc/RunIIFall17NanoAODv5/QCD_HT700to1000_TuneCP5_13TeV-madgraph-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano1June2019_102X_mc2017_realistic_v7-v1/50000/204CD301-86BC-2B45-B2AE-899D1072E55F.root
/store/mc/RunIIFall17NanoAODv5/QCD_HT700to1000_TuneCP5_13TeV-madgraph-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano1June2019_102X_mc2017_realistic_v7-v1/50000/F9AD22A0-B8D3-494B-9CD3-87E4CEE9BDD2.root
/store/mc/RunIIFall17NanoAODv5/QCD_HT700to1000_TuneCP5_13TeV-madgraph-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano1June2019_102X_mc2017_realistic_v7-v1/50000/F959C382-674D-EF43-ADB0-E1E50E4EE54B.root
/store/mc/RunIIFall17NanoAODv5/QCD_HT700to1000_TuneCP5_13TeV-madgraph-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano1June2019_102X_mc2017_realistic_v7-v1/50000/5AFCEE9C-2C2A-D147-A9BA-2101C9E00957.root
/store/mc/RunIIFall17NanoAODv5/QCD_HT700to1000_TuneCP5_13TeV-madgraph-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano1June2019_102X_mc2017_realistic_v7-v1/50000/C3E88B2C-6967-1044-AF23-B43222DDF387.root
/store/mc/RunIIFall17NanoAODv5/QCD_HT700to1000_TuneCP5_13TeV-madgraph-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano1June2019_102X_mc2017_realistic_v7-v1/50000/642478FE-E9BB-3746-93F9-319BF71F7DE4.root
/store/mc/RunIIFall17NanoAODv5/QCD_HT700to1000_TuneCP5_13TeV-madgraph-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano1June2019_102X_mc2017_realistic_v7-v1/50000/D589046D-4394-4E4A-8869-CC1BF3EFC0AE.root
/store/mc/RunIIFall17NanoAODv5/QCD_HT700to1000_TuneCP5_13TeV-madgraph-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano1June2019_102X_mc2017_realistic_v7-v1/50000/861531B1-4A56-9A40-B5CF-68E6A0078C76.root
/store/mc/RunIIFall17NanoAODv5/QCD_HT700to1000_TuneCP5_13TeV-madgraph-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano1June2019_102X_mc2017_realistic_v7-v1/50000/CC0D11AD-31CB-C64A-A967-10D13A28A06F.root
/store/mc/RunIIFall17NanoAODv5/QCD_HT700to1000_TuneCP5_13TeV-madgraph-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano1June2019_102X_mc2017_realistic_v7-v1/50000/113F9B01-037E-1949-806B-492DE08F9C7C.root
/store/mc/RunIIFall17NanoAODv5/QCD_HT700to1000_TuneCP5_13TeV-madgraph-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano1June2019_102X_mc2017_realistic_v7-v1/50000/52FAA999-1549-B04C-A7FE-F402982814C7.root
/store/mc/RunIIFall17NanoAODv5/QCD_HT700to1000_TuneCP5_13TeV-madgraph-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano1June2019_102X_mc2017_realistic_v7-v1/50000/132B1CB9-B9CC-6842-A3F6-4743D83310C1.root""".split()

nano_qcdht_700to1000 = ['root://cmsxrootd.fnal.gov/' + lfn for lfn in nano_qcdht_700to1000]

In [3]:
nano_zjets_600to800_raw = """/store/mc/RunIIFall17NanoAODv5/ZJetsToQQ_HT600to800_qc19_4j_TuneCP5_13TeV-madgraphMLM-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano1June2019_102X_mc2017_realistic_v7-v1/110000/6900A7A3-738B-3C42-8A26-140F53E8BFE2.root
/store/mc/RunIIFall17NanoAODv5/ZJetsToQQ_HT600to800_qc19_4j_TuneCP5_13TeV-madgraphMLM-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano1June2019_102X_mc2017_realistic_v7-v1/110000/495E9E02-9027-6045-B643-C23F6036C68A.root
/store/mc/RunIIFall17NanoAODv5/ZJetsToQQ_HT600to800_qc19_4j_TuneCP5_13TeV-madgraphMLM-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano1June2019_102X_mc2017_realistic_v7-v1/120000/28DA13A1-D782-2846-955D-FD507D2F3032.root
/store/mc/RunIIFall17NanoAODv5/ZJetsToQQ_HT600to800_qc19_4j_TuneCP5_13TeV-madgraphMLM-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano1June2019_102X_mc2017_realistic_v7-v1/120000/DA91E0D7-98C5-5C41-A826-34D23A481F91.root
/store/mc/RunIIFall17NanoAODv5/ZJetsToQQ_HT600to800_qc19_4j_TuneCP5_13TeV-madgraphMLM-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano1June2019_102X_mc2017_realistic_v7-v1/120000/00C16207-80F2-9B44-998C-F43DA013A656.root
/store/mc/RunIIFall17NanoAODv5/ZJetsToQQ_HT600to800_qc19_4j_TuneCP5_13TeV-madgraphMLM-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano1June2019_102X_mc2017_realistic_v7-v1/120000/F788BB5F-C2BE-5643-B671-9C8136C21970.root
/store/mc/RunIIFall17NanoAODv5/ZJetsToQQ_HT600to800_qc19_4j_TuneCP5_13TeV-madgraphMLM-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano1June2019_102X_mc2017_realistic_v7-v1/60000/4C373CDE-24AD-874A-BC53-C4C5A1B05BDB.root
/store/mc/RunIIFall17NanoAODv5/ZJetsToQQ_HT600to800_qc19_4j_TuneCP5_13TeV-madgraphMLM-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano1June2019_102X_mc2017_realistic_v7-v1/100000/9F20AC00-01F5-7644-94D7-4058AD751A8D.root
/store/mc/RunIIFall17NanoAODv5/ZJetsToQQ_HT600to800_qc19_4j_TuneCP5_13TeV-madgraphMLM-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano1June2019_102X_mc2017_realistic_v7-v1/100000/E2F82617-AB1C-EF4A-9E08-777519C32DC4.root
/store/mc/RunIIFall17NanoAODv5/ZJetsToQQ_HT600to800_qc19_4j_TuneCP5_13TeV-madgraphMLM-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano1June2019_102X_mc2017_realistic_v7-v1/100000/FB02D1F2-F90F-F545-90CE-A5A7ECF937E4.root
/store/mc/RunIIFall17NanoAODv5/ZJetsToQQ_HT600to800_qc19_4j_TuneCP5_13TeV-madgraphMLM-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano1June2019_102X_mc2017_realistic_v7-v1/100000/5691F9FB-C72E-504F-BB62-62243E157006.root
/store/mc/RunIIFall17NanoAODv5/ZJetsToQQ_HT600to800_qc19_4j_TuneCP5_13TeV-madgraphMLM-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano1June2019_102X_mc2017_realistic_v7-v1/100000/54621C6F-A1D2-0C4F-B524-051303A9CC7A.root
/store/mc/RunIIFall17NanoAODv5/ZJetsToQQ_HT600to800_qc19_4j_TuneCP5_13TeV-madgraphMLM-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano1June2019_102X_mc2017_realistic_v7-v1/100000/41C4F2A9-ED64-5949-9C87-2A75D03BC518.root
/store/mc/RunIIFall17NanoAODv5/ZJetsToQQ_HT600to800_qc19_4j_TuneCP5_13TeV-madgraphMLM-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano1June2019_102X_mc2017_realistic_v7-v1/100000/6F47D188-6607-5945-A031-DB6DD9F28441.root
/store/mc/RunIIFall17NanoAODv5/ZJetsToQQ_HT600to800_qc19_4j_TuneCP5_13TeV-madgraphMLM-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano1June2019_102X_mc2017_realistic_v7-v1/100000/F43E2636-4674-4446-8254-8AD6547CE4BD.root
/store/mc/RunIIFall17NanoAODv5/ZJetsToQQ_HT600to800_qc19_4j_TuneCP5_13TeV-madgraphMLM-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano1June2019_102X_mc2017_realistic_v7-v1/60000/4465CB11-6E04-0B43-BDBA-D9A165058A22.root""".split()

nano_zjets_600to800 = list()
for i, lfn in enumerate(nano_zjets_600to800_raw):
    #if i == 4 or i == 13 or i == 14:
    #    print(lfn)
    #    continue
    nano_zjets_600to800.append('root://cmsxrootd.fnal.gov/' + lfn)
print(len(nano_zjets_600to800))


16


In [4]:
import pyspark.sql.functions as fn
from tqdm import tqdm
import json

#central nano
#/QCD_HT700to1000_TuneCP5_13TeV-madgraph-pythia8/RunIIFall17NanoAODv5-PU2017_12Apr2018_Nano1June2019_new_pmx_102X_mc2017_realistic_v7-v1/NANOAODSIM

datasets = {'QCD_HT700to1000_TuneCP5_13TeV-madgraph-pythia8': {'files': nano_qcdht_700to1000,
                                                               'treename': 'Events'},
            'QCD_HT1000to1500_TuneCP5_13TeV-madgraph-pythia8': {'files': ['root://cmseos.fnal.gov//eos/uscms/store/user/ncsmith/pancakes/01/QCD_HT1000to1500_TuneCP5_13TeV-madgraph-pythia8/pancakes-01_RunIIFall17MiniAODv2-PU2017_12Apr2018_new_pmx_94X_v14-v1/'],
                                                                'treename': 'Events'},
            'ZJetsToQQ_HT600to800_qc19_4j_TuneCP5_13TeV': {'files': ['root://cmseos.fnal.gov//eos/uscms/store/user/ncsmith/pancakes/01/ZJetsToQQ_HT600to800_qc19_4j_TuneCP5_13TeV-madgraphMLM-pythia8/pancakes-01_RunIIFall17MiniAODv2-PU2017_12Apr2018_94X_v14-v1'],
                                                           'treename': 'Events'},
            'ZJetsToQQ_HT600to800_qc19_4j_TuneCP5_13TeV_nano': {'files': nano_zjets_600to800,
                                                                'treename': 'Events'}
            }

all_bits = {}
with open('metadata/samples_allyears.json') as f:
    temp = json.load(f)
    all_bits = temp['Hbb_create_2017']
bits_datasets = {}
for ds, flist in all_bits.items():
    if (#ds != 'QCD_HT700to1000_TuneCP5_13TeV-madgraphMLM-pythia8' and
        #ds != 'QCD_HT1000to1500_TuneCP5_13TeV-madgraphMLM-pythia8' and
        ds != 'ZJetsToQQ_HT600to800_qc19_4j_TuneCP5_13TeV'): continue
    if '/skim/' in flist[0]:
        bits_datasets[ds] = {'files': flist, 'treename': 'otree'}
    else:
        bits_datasets[ds] = {'files': flist, 'treename': 'Events'}

In [5]:
print(bits_datasets)

{'ZJetsToQQ_HT600to800_qc19_4j_TuneCP5_13TeV': {'files': ['root://cmseos.fnal.gov//eos/uscms/store/user/lpcbacon/dazsle/zprimebits-v15.04-vbf/skim/ZJetsToQQ_HT600to800_qc19_4j_TuneCP5_13TeV_0.root', 'root://cmseos.fnal.gov//eos/uscms/store/user/lpcbacon/dazsle/zprimebits-v15.04-vbf/skim/ZJetsToQQ_HT600to800_qc19_4j_TuneCP5_13TeV_1.root', 'root://cmseos.fnal.gov//eos/uscms/store/user/lpcbacon/dazsle/zprimebits-v15.04-vbf/skim/ZJetsToQQ_HT600to800_qc19_4j_TuneCP5_13TeV_10.root', 'root://cmseos.fnal.gov//eos/uscms/store/user/lpcbacon/dazsle/zprimebits-v15.04-vbf/skim/ZJetsToQQ_HT600to800_qc19_4j_TuneCP5_13TeV_11.root', 'root://cmseos.fnal.gov//eos/uscms/store/user/lpcbacon/dazsle/zprimebits-v15.04-vbf/skim/ZJetsToQQ_HT600to800_qc19_4j_TuneCP5_13TeV_2.root', 'root://cmseos.fnal.gov//eos/uscms/store/user/lpcbacon/dazsle/zprimebits-v15.04-vbf/skim/ZJetsToQQ_HT600to800_qc19_4j_TuneCP5_13TeV_3.root', 'root://cmseos.fnal.gov//eos/uscms/store/user/lpcbacon/dazsle/zprimebits-v15.04-vbf/skim/ZJets

In [6]:
fatJetNano = 'CustomAK8Puppi' #FatJet

In [7]:
df_pcakes = spark.read.format('root') \
                 .option('tree','Events') \
                 .option('partitionSize','200000') \
                 .load(datasets['ZJetsToQQ_HT600to800_qc19_4j_TuneCP5_13TeV']['files'])

pcakes_columns = ['run', 'luminosityBlock', 'event',
                  'genWeight',
                  'Pileup_nPU', 'Pileup_nTrueInt',
                  'Generator_x1','Generator_x2',
                  f'n{fatJetNano}', 
                  f'{fatJetNano}_pt', f'{fatJetNano}_eta', f'{fatJetNano}_phi', f'{fatJetNano}_mass',
                  f'{fatJetNano}_rawFactor',
                  f'{fatJetNano}_msoftdrop',
                  f'{fatJetNano}_btagDDBvL',f'{fatJetNano}_btagDDCvB', f'{fatJetNano}_btagDDCvL',
                  'nMuon', 'Muon_pt', 'Muon_eta', 'Muon_phi', 'Muon_mass',
                  'nElectron', 'Electron_pt', 'Electron_eta', 'Electron_phi', 'Electron_mass',
                  'nPhoton', 'Photon_pt', 'Photon_eta', 'Photon_phi', 'Photon_mass',
                  'nTau', 'Tau_pt', 'Tau_eta', 'Tau_phi', 'Tau_mass'                  
                  ]

df_pcakes = df_pcakes.select(*pcakes_columns)


df_nano = spark.read.format('root') \
                 .option('tree','Events') \
                 .option('partitionSize','200000') \
                 .load(datasets['ZJetsToQQ_HT600to800_qc19_4j_TuneCP5_13TeV_nano']['files'])

fatJetNano = 'FatJet'
nano_columns = ['run', 'luminosityBlock', 'event',
                  'genWeight',
                  'Pileup_nPU', 'Pileup_nTrueInt',
                  'Generator_x1','Generator_x2',
                  f'n{fatJetNano}', 
                  f'{fatJetNano}_pt', f'{fatJetNano}_eta', f'{fatJetNano}_phi', f'{fatJetNano}_mass',
                  f'{fatJetNano}_rawFactor',
                  f'{fatJetNano}_msoftdrop',
                  f'{fatJetNano}_btagDDBvL',f'{fatJetNano}_btagDDCvB', f'{fatJetNano}_btagDDCvL',
                  'nMuon', 'Muon_pt', 'Muon_eta', 'Muon_phi', 'Muon_mass',
                  'nElectron', 'Electron_pt', 'Electron_eta', 'Electron_phi', 'Electron_mass',
                  'nPhoton', 'Photon_pt', 'Photon_eta', 'Photon_phi', 'Photon_mass',
                  'nTau', 'Tau_pt', 'Tau_eta', 'Tau_phi', 'Tau_mass'                  
                  ]

df_nano = df_nano.select(*nano_columns)

for col in pcakes_columns:
    if col == 'run' or col =='luminosityBlock' or col == 'event': continue        
    df_nano = df_nano.withColumnRenamed(col, col + '_nano')

print(df_nano)
    
df_bits = spark.read.format('root') \
               .option('tree','otree') \
               .option('partitionSize','200000') \
               .load(bits_datasets['ZJetsToQQ_HT600to800_qc19_4j_TuneCP5_13TeV']['files'])
from columns import gghbbcolumns, gghbbcolumns_mc 
bits_id_columns = ['runNum', 'lumiSec', 'evtNum']
bits_columns = []
bits_columns.extend(bits_id_columns)
bits_columns.extend(gghbbcolumns)
bits_columns.extend(gghbbcolumns_mc)
bits_columns.append('evtWeight')
bits_columns.append('AK8Puppijet0_pt_old')

df_bits = df_bits.select(*bits_columns) \
                 .withColumnRenamed('runNum','run') \
                 .withColumnRenamed('lumiSec','luminosityBlock') \
                 .withColumnRenamed('evtNum','event') \
                 .withColumnRenamed('ntau','ntau_bits')

DataFrame[run: bigint, luminosityBlock: bigint, event: bigint, genWeight_nano: float, Pileup_nPU_nano: int, Pileup_nTrueInt_nano: float, Generator_x1_nano: float, Generator_x2_nano: float, nFatJet: int, FatJet_pt: array<float>, FatJet_eta: array<float>, FatJet_phi: array<float>, FatJet_mass: array<float>, FatJet_rawFactor: array<float>, FatJet_msoftdrop: array<float>, FatJet_btagDDBvL: array<float>, FatJet_btagDDCvB: array<float>, FatJet_btagDDCvL: array<float>, nMuon_nano: int, Muon_pt_nano: array<float>, Muon_eta_nano: array<float>, Muon_phi_nano: array<float>, Muon_mass_nano: array<float>, nElectron_nano: int, Electron_pt_nano: array<float>, Electron_eta_nano: array<float>, Electron_phi_nano: array<float>, Electron_mass_nano: array<float>, nPhoton_nano: int, Photon_pt_nano: array<float>, Photon_eta_nano: array<float>, Photon_phi_nano: array<float>, Photon_mass_nano: array<float>, nTau_nano: int, Tau_pt_nano: array<float>, Tau_eta_nano: array<float>, Tau_phi_nano: array<float>, Tau_m

In [ ]:
import uproot
import os

print(os.environ['X509_USER_PROXY'])
test_pcakes = uproot.open(datasets['QCD_HT700to1000_TuneCP5_13TeV-madgraph-pythia8']['files'][0])
test_bits = uproot.open(bits_datasets['QCD_HT700to1000_TuneCP5_13TeV-madgraphMLM-pythia8']['files'][0])

print(test_pcakes['Events'].keys())

print(test_bits['otree'].keys())

In [8]:
df_pcakes.count()

8882592

In [9]:
df_bits.count()

2599536

In [10]:
df_nano.count()

8882592

In [11]:
df_both = df_pcakes.join(df_nano, ['run', 'luminosityBlock', 'event'])
df_both = df_both.join(df_bits, ['run', 'luminosityBlock', 'event'])

In [12]:
df_both.count()

2599536

In [ ]:
interesting_cols = ['run','luminosityBlock','event','Pileup_nTrueInt','Pileup_nPU',
                    'npu','evtWeight', 'scale1fb','genWeight']

df_both.select(*interesting_cols) \
       .withColumn('diff', df_both.Pileup_nTrueInt - df_both.npu) \
       .agg({"diff":"sum", "evtWeight":"sum", "scale1fb":"sum", "genWeight":"sum"}).collect()

In [ ]:
df_both.select(*interesting_cols).show()

In [13]:
out_dir = 'hdfs://cmshdfs.fnal.gov:8020/scratch/lagray/nanobits/ZJetsToQQ_HT600to800_qc19_4j_TuneCP5_13TeV'

In [ ]:
df_both.write.parquet(out_dir)

In [ ]:
df_from_pq = spark.read.parquet(out_dir)

interesting_cols = ['run','luminosityBlock','event','Pileup_nTrueInt','Pileup_nTrueInt_nano',
                    'npu']

df_from_pq.count()

In [ ]:
df_from_pq

In [ ]:
#get the hbb analysis worker
from coffea.util import load
from coffea.analysis_objects import JaggedCandidateArray as Cands
from coffea import processor, hist
import numpy as np
import warnings
import math




class BoostedHbbPancakesProcessor(processor.ProcessorABC):
    
    
    
    def __init__(self, corrections, columns=[], debug=False, year='2017', skipPileup=False, skipTrigger=False):
        self._columns = columns
        self._corrections = corrections
        self._debug = debug
        self._year = year
        self._skipPileup = skipPileup
        self._skipTrigger = skipTrigger
        
        dataset_axis = hist.Cat("dataset", "Primary dataset")
        gencat_axis = hist.Bin("AK8Puppijet0_isHadronicV", "V matching index", [0,1,2,3,9,10,11])
        jetpt_axis = hist.Bin("AK8Puppijet0_pt", r"Jet $p_T$", [450, 500, 550, 600, 675, 800, 1200])
        jetmass_axis = hist.Bin("AK8Puppijet0_msd", r"Jet $m_{sd}$", 23, 40, 201)
        jetpt_coarse_axis = hist.Bin("AK8Puppijet0_pt", r"Jet $p_T$", [450, 600, 1200])
        jetmass_coarse_axis = hist.Bin("AK8Puppijet0_msd", r"Jet $m_{sd}$", [40, 82, 103, 152, 201])
        jetrho_axis = hist.Bin("ak8jet_rho", r"Jet $\rho$", 13, -6, -2.1)
        doubleb_axis = hist.Bin("AK8Puppijet0_deepdoubleb", "Double-b", 20, 0., 1)
        doublec_axis = hist.Bin("AK8Puppijet0_deepdoublec", "Double-c", 20, 0., 1.)
        doublecvb_axis = hist.Bin("AK8Puppijet0_deepdoublecvb", "Double-cvb", 20, 0., 1.)
        doubleb_wps = [1., 0.9, 0.89, 0.85, 0.7]
        doubleb_coarse_axis = hist.Bin("AK8Puppijet0_deepdoubleb", "Double-b", doubleb_wps[::-1])
        doublec_wps = [0.87, 0.84, 0.83, 0.79, 0.69]
        doublec_coarse_axis = hist.Bin("AK8Puppijet0_deepdoublec", "Double-c", doublec_wps[::-1])
        doublecvb_wps = [0.93, 0.91, 0.6, 0.2, 0.17]
        doublecvb_coarse_axis = hist.Bin("AK8Puppijet0_deepdoublecvb", "Double-cvb", doublecvb_wps[::-1])
        
        hists = processor.dict_accumulator()
        hist.Hist.DEFAULT_DTYPE = 'f'
        
        hists['sumw']     = processor.defaultdict_accumulator(int)
        hists['sumw_pos'] = processor.defaultdict_accumulator(int)
        hists['sumw_neg'] = processor.defaultdict_accumulator(int)
        hists['sumw_abs'] = processor.defaultdict_accumulator(int)
        hists['jetpt_preselection'] = hist.Hist("Events",
                                                dataset_axis,
                                                hist.Bin("AK8Puppijet0_pt", "Jet $p_T$", 50, 300, 1300),
                                                )
        
        hists['jeteta_preselection'] = hist.Hist("Events",
                                                 dataset_axis,
                                                 hist.Bin("AK8Puppijet0_eta", r"Jet $\eta$", 50, -3, 3),
                                                 )
        
        hists['jetmsd_preselection'] = hist.Hist("Events",
                                                dataset_axis,
                                                hist.Bin("AK8Puppijet0_msd", r"Jet $m_{sd}$", 46, 0, 414)
                                                )
        hists['jetddb_preselection'] = hist.Hist("Events",
                                                dataset_axis,
                                                hist.Bin("AK8Puppijet0_deepdoubleb", r"Jet deepdoubleb", 200, -2, 2)
                                                )
        hists['jetddc_preselection'] = hist.Hist("Events",
                                                dataset_axis,
                                                hist.Bin("AK8Puppijet0_deepdoublec", r"Jet deepdoublec", 200, -2, 2)
                                                )
        hists['jetddcvb_preselection'] = hist.Hist("Events",
                                                dataset_axis,
                                                hist.Bin("AK8Puppijet0_deepdoublecvb", r"Jet deepdoublecvb", 200, -2, 2)
                                                )
        
        hists['jetddb_matched'] = hist.Hist("Events",
                                                dataset_axis,
                                                hist.Bin("AK8Puppijet0_deepdoubleb", r"Jet deepdoubleb", 200, -2, 2)
                                                )
        hists['jetddc_matched'] = hist.Hist("Events",
                                                dataset_axis,
                                                hist.Bin("AK8Puppijet0_deepdoublec", r"Jet deepdoublec", 200, -2, 2)
                                                )
        hists['jetddcvb_matched'] = hist.Hist("Events",
                                                dataset_axis,
                                                hist.Bin("AK8Puppijet0_deepdoublecvb", r"Jet deepdoublecvb", 200, -2, 2)
                                                )
        
        hists['bitsddb_preselection'] = hist.Hist("Events",
                                                dataset_axis,
                                                hist.Bin("AK8Puppijet0_deepdoubleb", r"Jet deepdoubleb", 200, -2, 2)
                                                )
        hists['bitsddc_preselection'] = hist.Hist("Events",
                                                dataset_axis,
                                                hist.Bin("AK8Puppijet0_deepdoublec", r"Jet deepdoublec", 200, -2, 2)
                                                )
        hists['bitsddcvb_preselection'] = hist.Hist("Events",
                                                dataset_axis,
                                                hist.Bin("AK8Puppijet0_deepdoublecvb", r"Jet deepdoublecvb", 200, -2, 2)
                                                )
        
        hists['bitsddb_matched'] = hist.Hist("Events",
                                                dataset_axis,
                                                hist.Bin("AK8Puppijet0_deepdoubleb", r"Jet deepdoubleb", 200, -2, 2)
                                                )
        hists['bitsddc_matched'] = hist.Hist("Events",
                                                dataset_axis,
                                                hist.Bin("AK8Puppijet0_deepdoublec", r"Jet deepdoublec", 200, -2, 2)
                                                )
        hists['bitsddcvb_matched'] = hist.Hist("Events",
                                                dataset_axis,
                                                hist.Bin("AK8Puppijet0_deepdoublecvb", r"Jet deepdoublecvb", 200, -2, 2)
                                                )
        
        def make_deltas(prefix, hists):
            hists['%s_dphi' % prefix] = hist.Hist("Events",
                                             dataset_axis,
                                             hist.Bin("delta_phi", "$\Delta\phi$", 100, -0.2, 0.2),
                                             )
            hists['%s_deta' % prefix] = hist.Hist("Events",
                                             dataset_axis,
                                             hist.Bin("delta_eta", "$\Delta\eta$", 100, -0.2, 0.2),
                                             )
            hists['%s_dpt' % prefix]  = hist.Hist("Events",
                                             dataset_axis,
                                             hist.Bin("delta_pt", "$\Delta p_T / p_T$", 400, -2, 2),
                                             )
        
            hists['%s_dmsd' % prefix]  = hist.Hist("Events",
                                             dataset_axis,
                                             hist.Bin("delta_msd", "$\Delta m_{sd}$ (GeV)", 400, -50, 50),
                                             )
        
            hists['%s_dddb' % prefix]  = hist.Hist("Events",
                                             dataset_axis,
                                             hist.Bin("delta_ddb", "$\Delta$ deepdoubleb", 400, -2, 2),
                                             )
        
            hists['%s_dddc' % prefix]  = hist.Hist("Events",
                                             dataset_axis,
                                             hist.Bin("delta_ddc", "$\Delta$ deepdoublec", 400, -2, 2),
                                             )
        
            hists['%s_dddcvb' % prefix]  = hist.Hist("Events",
                                             dataset_axis,
                                             hist.Bin("delta_ddcvb", "$\Delta$ deepdoublecvb", 400, -2, 2),
                                             )
            
            hists['%s_ddbcor' % prefix] = hist.Hist("Events",
                                                dataset_axis,
                                                hist.Bin("AK8Puppijet0_deepdoubleb_first", r"Jet deepdoubleb 1", 200, 0, 1),
                                                hist.Bin("AK8Puppijet0_deepdoubleb_second", r"Jet deepdoubleb 2", 200, 0, 1)
                                                )
            
            hists['%s_ddccor' % prefix] = hist.Hist("Events",
                                                dataset_axis,
                                                hist.Bin("AK8Puppijet0_deepdoublec_first", r"Jet deepdoublec 1", 200, 0, 1),
                                                hist.Bin("AK8Puppijet0_deepdoublec_second", r"Jet deepdoublec 2", 200, 0, 1)
                                                )
            
            hists['%s_ddcvbcor' % prefix] = hist.Hist("Events",
                                                dataset_axis,
                                                hist.Bin("AK8Puppijet0_deepdoublecvb_first", r"Jet deepdoublecvb 1", 200, 0, 1),
                                                hist.Bin("AK8Puppijet0_deepdoublecvb_second", r"Jet deepdoublecvb 2", 200, 0, 1)
                                                )
            
        
        make_deltas('pcakesbits',hists)
        make_deltas('nanobits',hists)
        make_deltas('pcakesnano',hists)

        
        self._accumulator = hists
    
    @property
    def columns(self):
        return self._columns

    @property
    def accumulator(self):
        return self._accumulator
    
    def process(self, df):
        dataset = df['dataset']
        if self._debug:
            print("Processing dataframe from", dataset)
        isRealData = dataset in ["JetHT", "SingleMuon", "data_obs_mu", "data_obs_jet"]
        
        fatJetName_nano = 'CustomAK8Puppi'
        fatJetName_bits = 'AK8Puppijet0'
        
        ak8_jets_pcakes = Cands.candidatesfromcounts(df['n%s' % fatJetName_nano],
                                                     pt=df['%s_pt' % fatJetName_nano],
                                                     eta=df['%s_eta' % fatJetName_nano],
                                                     phi=df['%s_phi' % fatJetName_nano],
                                                     mass=df['%s_mass' % fatJetName_nano],
                                                     toRaw=df['%s_rawFactor' % fatJetName_nano],
                                                     msd=df['%s_msoftdrop' % fatJetName_nano],
                                                     deepdoubleb = df['%s_btagDDBvL' % fatJetName_nano],
                                                     deepdoublec = df['%s_btagDDCvL' % fatJetName_nano],
                                                     deepdoublecvb = df['%s_btagDDCvB' % fatJetName_nano])    
    
        fatJetName_nano = 'FatJet'
        ak8_jets_nano = Cands.candidatesfromcounts(df['n%s' % fatJetName_nano],
                                                   pt=df['%s_pt' % fatJetName_nano],
                                                   eta=df['%s_eta' % fatJetName_nano],
                                                   phi=df['%s_phi' % fatJetName_nano],
                                                   mass=df['%s_mass' % fatJetName_nano],
                                                   toRaw=df['%s_rawFactor' % fatJetName_nano],
                                                   msd=df['%s_msoftdrop' % fatJetName_nano],
                                                   deepdoubleb = df['%s_btagDDBvL' % fatJetName_nano],
                                                   deepdoublec = df['%s_btagDDCvL' % fatJetName_nano],
                                                   deepdoublecvb = df['%s_btagDDCvB' % fatJetName_nano])
        
        ak8_jets_bits = Cands.candidatesfromcounts(np.ones_like(df['n%s' % fatJetName_nano]),
                                                   pt=df['%s_pt_old' % fatJetName_bits],
                                                   eta=df['%s_eta' % fatJetName_bits],
                                                   phi=df['%s_phi' % fatJetName_bits],
                                                   mass=df['%s_msd' % fatJetName_bits],
                                                   msd=df['%s_msd' % fatJetName_bits],
                                                   deepdoubleb = df['%s_deepdoubleb' % fatJetName_bits],
                                                   deepdoublec = df['%s_deepdoublec' % fatJetName_bits],
                                                   deepdoublecvb = df['%s_deepdoublecvb' % fatJetName_bits]
                                                   )
        
        leading_jet_idxs = ak8_jets_pcakes.pt.argmax()
        leading_jets = ak8_jets_pcakes[leading_jet_idxs]
        
        leading_jet_idxs_nano = ak8_jets_nano.pt.argmax()
        leading_jets_nano = ak8_jets_nano[leading_jet_idxs_nano]
        
        
        clean_bits_jets = ak8_jets_bits[ak8_jets_bits.pt > 0.0]
        
        if self._debug:
            print('leading jets pt:',leading_jets.pt)
            print('leading jets eta:',leading_jets.eta)
        
        hout = self.accumulator.identity()
        unit_wgts = np.sign(df['genWeight'])
        pos_wgts = unit_wgts > 0
        hout['sumw'][dataset] += unit_wgts.sum()
        hout['sumw_pos'][dataset] += unit_wgts[pos_wgts].sum()
        hout['sumw_neg'][dataset] += unit_wgts[~pos_wgts].sum()
        hout['sumw_abs'][dataset] += np.abs(unit_wgts).sum()
        hout['jetpt_preselection'].fill(dataset=dataset, AK8Puppijet0_pt=leading_jets.pt.flatten())
        hout['jeteta_preselection'].fill(dataset=dataset, AK8Puppijet0_eta=leading_jets.eta.flatten())
        hout['jetmsd_preselection'].fill(dataset=dataset, AK8Puppijet0_msd=leading_jets.msd.flatten())
        hout['jetddb_preselection'].fill(dataset=dataset, AK8Puppijet0_deepdoubleb=leading_jets.deepdoubleb.flatten())
        hout['jetddc_preselection'].fill(dataset=dataset, AK8Puppijet0_deepdoublec=leading_jets.deepdoublec.flatten())
        hout['jetddcvb_preselection'].fill(dataset=dataset, AK8Puppijet0_deepdoublecvb=leading_jets.deepdoublecvb.flatten())
        
        hout['bitsddb_preselection'].fill(dataset=dataset, AK8Puppijet0_deepdoubleb=clean_bits_jets.deepdoubleb.flatten())
        hout['bitsddc_preselection'].fill(dataset=dataset, AK8Puppijet0_deepdoublec=clean_bits_jets.deepdoublec.flatten())
        hout['bitsddcvb_preselection'].fill(dataset=dataset, AK8Puppijet0_deepdoublecvb=clean_bits_jets.deepdoublecvb.flatten())
            
        #try matching to pcakes data
        has_bits = np.abs(ak8_jets_bits.pt.content) > 0.0
        ak8_bits = ak8_jets_bits[has_bits]
        ak8_pcakes = ak8_jets_pcakes[has_bits]
        ak8_nano = ak8_jets_nano[has_bits]
        
        matches = ak8_pcakes.fastmatch(ak8_bits, deltaRCut=0.4)
        pcakes_matches = ak8_pcakes[matches][matches.any()]
        
        matches_nano = ak8_nano.fastmatch(ak8_bits, deltaRCut=0.4)
        nano_matches = ak8_nano[matches_nano][matches_nano.any()]
        
        bits_matches = ak8_bits[matches.any()]        
        bits_matches_nano = ak8_bits[matches_nano.any()]
        
        matches_pcakesnano = leading_jets.fastmatch(leading_jets_nano, deltaRCut=0.4)
        pcakesnano_matches = leading_jets[matches_pcakesnano]

        
        if self._debug:
            print('pcakes matches',pcakes_matches.size, pcakes_matches.counts.sum())
            print('pcakes matches',pcakes_matches.size, pcakes_matches.counts.sum())
            print('bits matches',bits_matches.size)
        bits_pcakes_pairs = pcakes_matches.cross(bits_matches) 
        bits_nano_pairs = nano_matches.cross(bits_matches_nano)
        pcakes_nano_pairs = leading_jets_nano.cross(pcakesnano_matches)
                
        hout['jetddb_matched'].fill(dataset=dataset, AK8Puppijet0_deepdoubleb=bits_pcakes_pairs.i0.deepdoubleb.flatten())
        hout['jetddc_matched'].fill(dataset=dataset, AK8Puppijet0_deepdoublec=bits_pcakes_pairs.i0.deepdoublec.flatten())
        hout['jetddcvb_matched'].fill(dataset=dataset, AK8Puppijet0_deepdoublecvb=bits_pcakes_pairs.i0.deepdoublecvb.flatten())
        
        hout['bitsddb_matched'].fill(dataset=dataset, AK8Puppijet0_deepdoubleb=bits_pcakes_pairs.i1.deepdoubleb.flatten())
        hout['bitsddc_matched'].fill(dataset=dataset, AK8Puppijet0_deepdoublec=bits_pcakes_pairs.i1.deepdoublec.flatten())
        hout['bitsddcvb_matched'].fill(dataset=dataset, AK8Puppijet0_deepdoublecvb=bits_pcakes_pairs.i1.deepdoublecvb.flatten())
        
         
        
        def fill_deltas(prefix,thepairs,hists):
            delta_pt    = (thepairs.i0.pt - thepairs.i1.pt)/thepairs.i1.pt
            delta_eta   = thepairs.i0.eta - thepairs.i1.eta
            delta_phi   = thepairs.i0.phi - thepairs.i1.phi
            delta_msd   = thepairs.i0.msd - thepairs.i1.msd
            delta_ddb   = thepairs.i0.deepdoubleb - thepairs.i1.deepdoubleb
            delta_ddc   = thepairs.i0.deepdoublec - thepairs.i1.deepdoublec
            delta_ddcvb = thepairs.i0.deepdoublecvb - thepairs.i1.deepdoublecvb       
        
            hists['%s_ddbcor' % prefix].fill(dataset=dataset,
                                             AK8Puppijet0_deepdoubleb_first = thepairs.i0.deepdoubleb.flatten(),
                                             AK8Puppijet0_deepdoubleb_second = thepairs.i1.deepdoubleb.flatten(),
                                             )
            
            hists['%s_ddccor' % prefix].fill(dataset=dataset,
                                             AK8Puppijet0_deepdoublec_first = thepairs.i0.deepdoublec.flatten(),
                                             AK8Puppijet0_deepdoublec_second = thepairs.i1.deepdoublec.flatten(),
                                             )
            
            hists['%s_ddcvbcor' % prefix].fill(dataset=dataset,
                                             AK8Puppijet0_deepdoublecvb_first = thepairs.i0.deepdoublecvb.flatten(),
                                             AK8Puppijet0_deepdoublecvb_second = thepairs.i1.deepdoublecvb.flatten(),
                                             )
        
            if self._debug:
                print('matched jets deta:',delta_phi)
                print('matched jets dphi:',delta_eta)
        
            hists['%s_deta' % prefix].fill(dataset=dataset, delta_eta=delta_eta.flatten())
            hists['%s_dphi' % prefix].fill(dataset=dataset, delta_phi=delta_phi.flatten())
            hists['%s_dpt' % prefix].fill(dataset=dataset, delta_pt=delta_pt.flatten())
            hists['%s_dmsd' % prefix].fill(dataset=dataset, delta_msd=delta_msd.flatten())
            hists['%s_dddb' % prefix].fill(dataset=dataset, delta_ddb=delta_ddb.flatten())
            hists['%s_dddc' % prefix].fill(dataset=dataset, delta_ddc=delta_ddc.flatten())
            hists['%s_dddcvb' % prefix].fill(dataset=dataset, delta_ddcvb=delta_ddcvb.flatten())
                
        fill_deltas('pcakesbits',bits_pcakes_pairs,hout)
        fill_deltas('nanobits',bits_nano_pairs,hout)
        fill_deltas('pcakesnano',pcakes_nano_pairs,hout)
        
        if self._debug:
                print(hout)
                
        return hout
        
    def postprocess(self, accumulator):
        # set everything to 1/fb scale
        lumi = 1000  # [1/pb]
        
        if 'sumw_external' in self._corrections:
            normlist = self._corrections['sumw_external']
            for key in accumulator['sumw'].keys():
                accumulator['sumw'][key] = normlist[key].value

        scale = {}
        for dataset, dataset_sumw in accumulator['sumw'].items():
            if dataset in self._corrections['xsections']:
                scale[dataset] = lumi*self._corrections['xsections'][dataset]/dataset_sumw
            else:
                warnings.warn("Missing cross section for dataset %s.  Normalizing to 1 pb" % dataset, RuntimeWarning)
                scale[dataset] = lumi / dataset_sumw
            
        for h in accumulator.values():
            if isinstance(h, hist.Hist):
                h.scale(scale, axis="dataset")

        return accumulator

from columns import gghbbcolumns, gghbbcolumns_mc
fatJetNano = 'CustomAK8Puppi'
columns = ['genWeight', 'Pileup_nTrueInt',
           f'n{fatJetNano}',f'{fatJetNano}_pt',f'{fatJetNano}_eta',f'{fatJetNano}_phi',f'{fatJetNano}_mass',
           f'{fatJetNano}_rawFactor',f'{fatJetNano}_msoftdrop',
           f'{fatJetNano}_btagDDBvL',f'{fatJetNano}_btagDDCvB', f'{fatJetNano}_btagDDCvL']
fatJetNano = 'FatJet'
columns.extend([f'n{fatJetNano}',f'{fatJetNano}_pt',f'{fatJetNano}_eta',f'{fatJetNano}_phi',f'{fatJetNano}_mass',
                f'{fatJetNano}_rawFactor',f'{fatJetNano}_msoftdrop',
                f'{fatJetNano}_btagDDBvL',f'{fatJetNano}_btagDDCvB', f'{fatJetNano}_btagDDCvL'])
columns.extend(gghbbcolumns)
columns.extend(gghbbcolumns_mc)
columns.append('ntau_bits')
columns.append('AK8Puppijet0_pt_old')

print(columns)
corrections = {'xsections': {}}

processor_instance = BoostedHbbPancakesProcessor(corrections, columns, debug=True)

datasets_comp = {'ZJetsToQQ_HT600to800_qc19_4j_TuneCP5_13TeV' : 'hdfs://cmshdfs.fnal.gov:8020/scratch/lagray/nanobits/ZJetsToQQ_HT600to800_qc19_4j_TuneCP5_13TeV'}


In [ ]:
import time
from coffea.processor import run_spark_job
from coffea.processor.spark.spark_executor import spark_executor

tic = time.time()
final_accumulator = run_spark_job(datasets_comp, processor_instance, spark_executor, 
                                  spark=spark, partitionsize=partitionsize, thread_workers=thread_workers)
dt = time.time() - tic


In [ ]:
final_accumulator

In [ ]:
nevt = sum(spark_executor.counts.values())
print('processed:',nevt,'events')
print('total time: ',dt/60)
print('μs/evt', dt/nevt*1e6)
print('Mevt/s', nevt/dt/1e6)


In [ ]:
%matplotlib inline
from coffea.hist import plot

print(final_accumulator['sumw'])
print(final_accumulator['sumw_pos'],final_accumulator['sumw_neg'])
print(final_accumulator['sumw_abs'])
fig, ax, _ = plot.plot1d(final_accumulator['nanobits_deta'], overlay='dataset')
ax.set_ylim([1e-5,50000])
ax.set_yscale('log')

In [ ]:
from matplotlib.colors import LogNorm
fig, ax, _ = plot.plot2d(final_accumulator['nanobits_ddbcor'].integrate('dataset'),
                         xaxis='AK8Puppijet0_deepdoubleb_first',
                         clear = True,
                         patch_opts={'norm': LogNorm()})
ax.set_title("(nano - bits)")
#plot.plot1d(final_accumulator['jetddb_preselection'], overlay='dataset', ax=ax)
#ax.set_ylim([1e-3,50000])
#ax.set_yscale('log')

In [ ]:
fig, ax, _ = plot.plotratio(num=final_accumulator['jetddb_matched'].integrate('dataset'),
                            denom=final_accumulator['bitsddb_matched'].integrate('dataset'),
                            error_opts={'color': 'k', 'marker': '.'},
                            unc='num')
ax.set_title('Nano / bits')

#plot.plot1d(final_accumulator['jetddb_preselection'], overlay='dataset', ax=ax)
ax.set_ylim([0,1.25])
#ax.set_yscale('log')

In [ ]:
from coffea import hist
from coffea.util import save
import gzip
import pickle
import numexpr
import numpy as np

nbins = sum(sum(arr.size for arr in h._sumw.values()) for h in final_accumulator.values() if isinstance(h, hist.Hist))
nfilled = sum(sum(np.sum(arr>0) for arr in h._sumw.values()) for h in final_accumulator.values() if isinstance(h, hist.Hist))
print("Processed %.1fM events" % (nevt/1e6, ))
print("Filled %.1fM bins" % (nbins/1e6, ))
print("Nonzero bins: %.1f%%" % (100*nfilled/nbins, ))

# Pickle is not very fast or memory efficient, will be replaced by something better soon
save(final_accumulator,'hists.coffea')

#dt = time.time() - tstart
#print("%.2f us*cpu/event overall" % (1e6*dt*nworkers/final_accumulators['nentries'], ))


In [ ]:
spark.stop()
